In [21]:
import pandas as pd
import pymorphy2

from tqdm import tqdm
tqdm.pandas()

import nltk
from nltk.tokenize import word_tokenize 
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from gensim.models import Phrases
# from skopt import BayesSearchCV
# import eli5

In [22]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Задание на первую лабораторную: сделать классификацию тональности коротких текстов
Датасет: https://github.com/sismetanin/rureviews 
Оставим только negative и positive отзывы
- почистить текст, убрать знаки пунктуации (+токенизация)
- лемматизировать текст 
- сгенерировать коллокации
- использовать логистическую регрессию
- поверх регрессии использовать GridSearchCV или BayesSearchCV

### Preprocess

In [23]:
df = pd.read_csv("women-clothing-accessories.3-class.balanced.csv", delimiter='\t')
# фильтруем создаем новый DataFrame, который содержит только те строки из df, 
df = df[df['sentiment'] != 'neautral']
# убрали знаки препинания
df['review_processed'] = df['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x)).values
# привели к нижнему регистру
df['review_processed'] = df['review_processed'].apply(lambda x: x.lower())
# разабъем слова
df['review_processed'] = df['review_processed'].apply(lambda x: word_tokenize(x))

# Создайте TF-IDF векторизатор (числовые признаки)
morph = pymorphy2.MorphAnalyzer()

df['review_morphed'] = df['review_processed'].progress_apply(lambda x: [morph.parse(word)[0].normal_form for word in x])


100%|██████████| 60000/60000 [02:42<00:00, 368.93it/s]


регресия

In [24]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X = tfidf_vectorizer.fit_transform([" ".join(x) for x in df['review_morphed']])

train_ds, test_ds, train_marks, test_marks = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)

reg = LogisticRegression(max_iter=1000)
reg.fit(train_ds, train_marks)
predict = reg.predict_log_proba(test_ds)[:, 1]

print('Метрика - ', roc_auc_score(test_marks, predict))

# eli5.show_weights(estimator=reg, 
#                   feature_names= list(vectorizer.get_feature_names()),
#                  top=(50, 50))

Метрика -  0.9735222897024753


# Grid search

регрессия с использованием метода Grid Search для нахождения наилучших гиперпараметров модели

In [25]:
param_grid = {
    'C': [3, 3.2, 3.4, 3.6, 4],
    'max_iter': [200, 400, 600, 1000]
}
# используем l2 регулизацию
reg_gsCV = LogisticRegression(penalty='l2')
gsCV = GridSearchCV(reg_gsCV, param_grid, verbose=True, n_jobs=8)
gsCV.fit(train_ds, train_marks)
predict = gsCV.predict_log_proba(test_ds)[:, 1]

print("Рос кривая - ", roc_auc_score(test_marks, predict))
print(gsCV.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Рос кривая -  0.9750521143633412
{'C': 3.2, 'max_iter': 200}


In [26]:
# пример
documents = ["Нью Йорк большой город", "Нью Йорк", "Поеду в Нью Йорк на следующей неделе"]
sentence_stream = [doc.split(" ") for doc in documents]
bigram = Phrases(sentence_stream, min_count=2, threshold=1)
sent = ['Он', 'поехал', 'в', 'Нью', 'Йорк']
print(bigram[sent])

['Он', 'поехал', 'в', 'Нью_Йорк']


сгенерировать коллокации

In [ ]:
# Колокации - связи между словами рядом стоящими
bigrams = Phrases(df['review_morphed'].values, min_count=2, threshold=1)
df['review_processed_coll'] = df['review_morphed'].apply(lambda x: bigrams[x])

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X = tfidf_vectorizer.fit_transform([" ".join(x) for x in df['review_processed_coll']])

train_ds, test_ds, train_marks, test_marks = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)

In [27]:
reg = LogisticRegression(max_iter=2000, n_jobs=-1)
reg.fit(train_ds, train_marks)
predict = reg.predict_log_proba(test_ds)[:, 1]

print('Рос кривая - ', roc_auc_score(test_marks, predict))
# eli5.show_weights(estimator=reg, 
#                   feature_names= list(vectorizer.get_feature_names()),
#                  top=(50, 50))

(60000, 73017)
Рос кривая 0.9726146496239656
